In [41]:
import pandas as pd
import numpy as np

In [42]:
df= pd.read_csv("/content/sample_data/diabetes.csv")

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [44]:
df.shape

(768, 9)

In [45]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [46]:
X= df.iloc[:,:-1].values
y= df.iloc[:,-1].values

In [47]:
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()

In [48]:
X = scaler.fit_transform(X)

In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test= train_test_split(X,y,test_size=0.2, random_state=42)

In [50]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [51]:
model= Sequential()
model.add(Dense(units=32,activation='relu',input_dim=X_train.shape[1]))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [52]:
model.fit(X_train,y_train, batch_size=32, epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.5833 - loss: 0.6703 - val_accuracy: 0.6364 - val_loss: 0.6354
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7087 - loss: 0.6125 - val_accuracy: 0.6948 - val_loss: 0.6003
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7531 - loss: 0.5561 - val_accuracy: 0.7273 - val_loss: 0.5768
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7185 - loss: 0.5590 - val_accuracy: 0.7532 - val_loss: 0.5569
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7427 - loss: 0.5299 - val_accuracy: 0.7727 - val_loss: 0.5445
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7660 - loss: 0.5104 - val_accuracy: 0.7727 - val_loss: 0.5364
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7633 - loss: 0.5032 - val_accuracy: 0.7792 - val_loss: 0.5302
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7508 - loss: 0.5076 - val_accuracy: 0.7792 - 

# 1.Select Appropriate optimizer
# 2. Select No. of nodes in layer
# 3. Select No. of layers
# 4. All in one model

In [53]:
!pip install -U keras-tuner
import keras_tuner as kt

In [54]:
def build_model(hp):
  model= Sequential()
  model.add(Dense(32,activation='relu',input_dim=X_train.shape[1]))
  model.add(Dense(1,activation='sigmoid'))

  optimizer=hp.Choice('optimizer', values=['adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [55]:
tuner= kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [56]:
tuner.search(X_train,y_train,epochs=6,validation_data=(X_test,y_test))

In [57]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [58]:
model= tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [59]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [60]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.7918 - loss: 0.4821 - val_accuracy: 0.7987 - val_loss: 0.5118
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7996 - loss: 0.4823 - val_accuracy: 0.7857 - val_loss: 0.5088
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8090 - loss: 0.4661 - val_accuracy: 0.7857 - val_loss: 0.5063
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7857 - loss: 0.4591 - val_accuracy: 0.7792 - val_loss: 0.5057
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7872 - loss: 0.4594 - val_accuracy: 0.7662 - val_loss: 0.5067
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7822 - loss: 0.4540 - val_accuracy: 0.7662 - val_loss: 0.5082
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7874 - loss: 0.4647 - val_accuracy: 0.7662 - val_loss: 0.5078
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8095 - loss: 0.4237 - val_accuracy: 0.76

In [61]:
def build_model(hp):
  model= Sequential()
  units=hp.Int('units',min_value=8,max_value=128,step=8)
  model.add(Dense(units=units,activation='relu',input_dim=X_train.shape[1]))
  model.add(Dense(units=1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [62]:
tuner= kt.GridSearch(build_model,
                     objective='val_accuracy',
                     max_trials=5,
                     directory='my_dir',
                     project_name='rmspropss')

Reloading Tuner from my_dir/rmspropss/tuner0.json


In [63]:
tuner.search(X_train,y_train,epochs=10,validation_data=(X_test,y_test))

In [64]:
tuner.get_best_hyperparameters()[0].values

{'units': 40}

In [65]:
model= tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [66]:
model.fit(X_train,y_train,batch_size=32,initial_epoch=6,epochs=100)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7700 - loss: 0.4789
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7800 - loss: 0.4612
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7508 - loss: 0.4807
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7796 - loss: 0.4483
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7704 - loss: 0.4584
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7891 - loss: 0.4473
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7873 - loss: 0.4575
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8002 - loss: 0.4385
Epoch 15/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7833 - loss: 0.4580
Epoch 16/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7733 - loss: 0.4525
Epoch 17/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7931 - loss: 0.4515
Epoch 18/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - ac

In [67]:
def build_models(hp):
  model= Sequential()

  model.add(Dense(8,activation='relu',input_dim=8))

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    model.add(Dense(8,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [68]:
kt.RandomSearch(build_models,
                objective='val_accuracy',
                max_trials=10,
                directory='my_dir',
                project_name='layer_nm')

In [69]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [70]:
tuner.get_best_hyperparameters()[0].values

{'units': 40}

In [77]:
def build_model(hp):
  model= Sequential()

  counter=0

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    if counter == 0:
      model.add(Dense(hp.Int('units'+str(i),min_value=8, max_value=128, step=8),activation=hp.Choice('activation',['relu','tanh','sigmoid']),input_dim=8))
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
      model.add(Dense(hp.Int('units'+str(i),min_value=8, max_value=128, step=8),activation=hp.Choice('activation',['relu','tanh','sigmoid'])))
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
      counter+=1

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer',['adam','nadam','rmsprop','adadelta','sgd']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
  return model

In [78]:
tuner= kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=10,
                       directory='my_dir',
                       project_name='final_prop')

In [79]:
tuner.search(X_train,y_train,epochs=10,validation_data=(X_test,y_test))

Trial 10 Complete [00h 00m 09s]
val_accuracy: 0.3571428656578064

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 01m 23s


In [80]:
model= tuner.get_best_models(num_models=1)[0]

In [82]:
model.fit(X_train,y_train,epochs=200,initial_epoch=10,validation_data=(X_test,y_test))

Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.6913 - loss: 0.5801 - val_accuracy: 0.7857 - val_loss: 0.5081
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7418 - loss: 0.5516 - val_accuracy: 0.7857 - val_loss: 0.5026
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6929 - loss: 0.5788 - val_accuracy: 0.7857 - val_loss: 0.4971
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7271 - loss: 0.5783 - val_accuracy: 0.7727 - val_loss: 0.4960
Epoch 15/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7062 - loss: 0.5485 - val_accuracy: 0.7597 - val_loss: 0.4924
Epoch 16/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7255 - loss: 0.5614 - val_accuracy: 0.7403 - val_loss: 0.4944
Epoch 17/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7195 - loss: 0.5557 - val_accuracy: 0.7403 - val_loss: 0.4953
Epoch 18/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7117 - loss: 0.5819 - val_accuracy: 0

In [81]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 6,
 'units0': 96,
 'activation': 'tanh',
 'dropout0': 0.1,
 'optimizer': 'sgd',
 'units1': 8,
 'dropout1': 0.1,
 'units2': 80,
 'dropout2': 0.2,
 'units3': 48,
 'dropout3': 0.8,
 'units4': 112,
 'dropout4': 0.3,
 'units5': 56,
 'dropout5': 0.5,
 'units6': 32,
 'dropout6': 0.9}